# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

A newer version of GraphLab Create (v1.8.1) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

[INFO] 1454629092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to C:\Users\Dmitrij\AppData\Local\Dato\Dato Launcher\lib\site-packages\certifi\cacert.pem
1454629092 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to dima.kost.94@mail.ru and will expire on November 29, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-6076 - Server binary: C:\Users\Dmitrij\AppData\Local\Dato\Dato Launcher\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Dmitrij\AppData\Local\Temp\graphlab_server_1454629092.log.0
[INFO] GraphLab Server Version: 1.7.1


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [71]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['intercept']=1;
    features = ['intercept']+features;
    features_sframe = data_sframe[features];
    
    feature_matrix = features_sframe.to_numpy();
    output_array = data_sframe[output];
    output_array = output_array.to_numpy();
    return (feature_matrix, output_array);

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [72]:
def predict_outcome(feature_matrix, weigths):
    return np.dot(feature_matrix,weigths);

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [73]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [74]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [75]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [76]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0);
    normalized_features = feature_matrix/norms;
    return (normalized_features, norms);

To test the function, run the following:

In [77]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [87]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [88]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [89]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [90]:
prediction = predict_outcome(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [91]:
def get_ro(feature, error, weidth):
    ro = np.dot(feature, (error+weidth*feature))
    return ro;


def get_prediction_error (prediction,output):
    return output-prediction;

def get_features_ro (initial_weidhts, output, feature_matrix):
    weights = initial_weidhts;
    features_ro = [];
    intercept = get_ro(feature_matrix[:,0], 
                        get_prediction_error( predict_outcome(feature_matrix, weights), output ), 
                        weights[0]);
    features_ro.append(intercept)
    for i in xrange(1,len(weights)):
        ro = get_ro(feature_matrix[:,i], 
                        get_prediction_error( predict_outcome(feature_matrix, weights), output ), 
                        weights[i]);
        features_ro.append(ro);
    return features_ro;



In [92]:
initial_ro = get_features_ro(weights, output, simple_feature_matrix);
print initial_ro
l1_1w = 2*initial_ro[1];
l1_2w = 2*initial_ro[2];
print l1_1w;
print l1_2w

[79400300.034929127, 87939470.772991031, 80966698.675965697]
175878941.546
161933397.352


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [93]:
l1_penalties =  np.logspace(1,7,num=13)

def quiz_1_question_check(weights):
    return weights[1] != 0 and weights[2] == 0;
    
def get_number_of_nnz(array):
    nnz_num = 0;
    for i in array: 
        if i!=0: 
            nnz_num+=1;
    return nnz_num;

def assist_l1_penalty(penalties, initial_weigths, features_matrix, callback, min_nonzeros):
    weights = initial_weigths;
    result_penalties = [];
    
    for penalty in penalties:
        #number of interation of coordinate dicent
        for i in range(100):
            #cycle throu all of the features except intercept 
            for  i in range(1,len(weights)):
                
                #modifaing the features based on ro parameter
                ro = get_ro(features_matrix[:,i], 
                            get_prediction_error( predict_outcome(features_matrix, weights), output ), 
                            weights[i]);
            
                if ro < - penalty/2:
                    weights[i] = ro + penalty/2;
            
                elif ro > penalty/2:
                    weights[i] = ro - penalty/2;
            
                else: weights[i] = 0;
        
        #asses weights on test function 
        if(callback(weights)): 
            result_penalties.append(penalty); 
        
    return result_penalties;

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [94]:
penalties = assist_l1_penalty(l1_penalties, weights, simple_feature_matrix, quiz_1_question_check, 1);
penalties;

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [95]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    
    # compute prediction
    prediction = predict_outcome(feature_matrix, weights);
    
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    error = output - prediction;
    ro_i = get_ro(feature_matrix[:,i], error, weights[i]);
    
    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2;
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2;
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [96]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [97]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    weights = initial_weights;
    not_converge = True;
    
    features_converged = 0;
    
    while features_converged != len(weights):
        features_converged = 0;
        for i in range(len(initial_weights)):
            
            old_weights_i = weights[i];
            
            weights[i] =  lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty);
            
            step_size = abs(old_weights_i - weights[i]);
    
            if(step_size > tolerance):
                continue;
            
            features_converged = features_converged + 1;
    return weights;

Using the following parameters, learn the weights on the sales dataset. 

In [98]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
simple_initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [99]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [102]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            simple_initial_weights, l1_penalty, tolerance)
print weights

[ 21624997.64707142  63157247.44345134         0.        ]


In [101]:
def get_model_rss(output,prediction):
    error = output - prediction;
    error_sqrt = np.dot(error, error);
    rss = error_sqrt.sum();
    return rss;

prediction = predict_outcome(normalized_simple_feature_matrix,weights);
rss = get_model_rss(output, prediction);
print rss

1.63049248148e+15


##QUIZ QUESTIONS
1. What is the RSS of the learned model on the normalized dataset? ***1630492474904516.0***
2. Which features had weight zero at convergence? *** It is 2 feature***

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [29]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [30]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']
output = 'price'

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [31]:
(multiple_feature_matrix, multiple_outout) = get_numpy_data(train_data, all_features, output);
(normalized_multiple_feature_matrix, norms) = normalize_features(multiple_feature_matrix);

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [53]:
l1_penalty = 1e7;
initial_weights = np.zeros(len(all_features)+1);
tolerance = 1;

weights_1e7 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix, 
                                                multiple_outout, initial_weights, l1_penalty, tolerance);
def print_weights(weights,features_names):
    print "intercept", weights[0]
    for i in range(len(features_names)):
        print features_names[i], weights[i+1];

print_weights(weights_1e7, all_features)

intercept 24429600.6093
bedrooms 0.0
bathrooms 0.0
sqft_living 48389174.3523
sqft_lot 0.0
floors 0.0
waterfront 3317511.16272
view 7329961.9849
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


##QUIZ QUESTION

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [33]:
l1_penalty=1e8
weights_1e8 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix, 
                                                multiple_outout, initial_weights, l1_penalty, tolerance);
print_weights(weights_1e7, all_features)

intercept 71114625.7528
bedrooms 0.0
bathrooms 0.0
sqft_living 0.0
sqft_lot 0.0
floors 0.0
waterfront 0.0
view 0.0
condition 0.0
grade 0.0
sqft_above 0.0
sqft_basement 0.0
yr_built 0.0
yr_renovated 0.0


##QUIZ QUESTION

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [41]:
l1_penalty=1e4;
tolerance=5e5

weights_1e4 = lasso_cyclical_coordinate_descent(normalized_multiple_feature_matrix, 
                                                multiple_outout, initial_weights, l1_penalty, tolerance);
print_weights(weights_1e4, all_features)

intercept 794582629.786
bedrooms -19532124.5632
bathrooms 13051054.1769
sqft_living 52143719.4942
sqft_lot -1348466.25048
floors 3834914.51996
waterfront 6863135.84843
view 4538296.44375
condition 8683558.99048
grade 128441512.586
sqft_above 0.0
sqft_basement 231472.975584
yr_built -905454330.94
yr_renovated 646689.324812


##QUIZ QUESTION

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [55]:
normalized_weight_1e7 = weights_1e7/norms;
normalized_weight_1e8 = weights_1e8/norms;
normalized_weight_1e4= weights_1e4/norms;
normalized_weight_1e7[3]

161.31745624837865

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [54]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [56]:
def get_model_rss(weights, data, output):
    predictions = predict_outcome(test_feature_matrix, weights);
    errors = output - predictions;
    errors_sqrt = errors**2;
    rss = errors.sum();
    return rss;

In [63]:
rss = [];
rss.append(get_model_rss(normalized_weight_1e4, test_feature_matrix, test_output));
rss.append(get_model_rss(normalized_weight_1e7, test_feature_matrix, test_output));
rss.append(get_model_rss(normalized_weight_1e8, test_feature_matrix, test_output));

In [64]:
rss.index(min(rss))

0

***QUIZ QUESTION***

Which model performed best on the test data?